In [1]:
## EDIT:
base_dir = '/glade/work/kjmayer/research/catalyst/TransferLearning/runmean_analysis/artificial_bias/perfectmodel_TLtest/'

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import random
import pandas as pd
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from tensorflow import keras
from tensorflow.keras import layers
import time

import sys
sys.path.append(base_dir)
from utils import split_cesm_retrain, plot_results
from exp_hp import get_hp

2024-03-20 09:43:46.134328: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 09:43:47.701601: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Num GPUs Available:  0


### Fuctions

In [3]:
def create_ann2(input_shape,output_shape, hiddens, RIDGE = 0, dropout = True, dropout_rate = 0.0):
    tf.keras.backend.clear_session() 

    # ----- input of cnn -----
    ann_input = keras.Input(shape = input_shape)

    for l,layer in enumerate(hiddens):
        # ----- ann layers -----
        if l == 0:
            x = tf.keras.layers.Dense(layer, 
                                      activation = 'relu',
                                      use_bias = True, 
                                      kernel_regularizer = tf.keras.regularizers.l1_l2(l1=0.0, l2=RIDGE),
                                      bias_initializer = tf.keras.initializers.RandomNormal(seed=SEED),
                                      kernel_initializer = tf.keras.initializers.RandomNormal(seed=SEED)
                                      )(ann_input)
            if dropout:
                x = layers.Dropout(rate = dropout_rate)(x)
        else:
            x = tf.keras.layers.Dense(layer, 
                                      activation = 'relu',
                                      use_bias = True, 
                                      kernel_regularizer = tf.keras.regularizers.l1_l2(l1=0.0, l2=0.0),
                                      bias_initializer = tf.keras.initializers.RandomNormal(seed=SEED),
                                      kernel_initializer = tf.keras.initializers.RandomNormal(seed=SEED)
                                      )(x)
        
    # ----- output -----
    ann_output = tf.keras.layers.Dense(output_shape,
                                       activation = tf.keras.activations.softmax,
                                       use_bias = True,
                                       kernel_regularizer = tf.keras.regularizers.l1_l2(l1=0.0, l2=0.0),
                                       bias_initializer = tf.keras.initializers.RandomNormal(seed=SEED),
                                       kernel_initializer = tf.keras.initializers.RandomNormal(seed=SEED)
                                      )(x)
    # ----- create unet -----
    ann = keras.Model(ann_input, ann_output, name = 'ann')
    
    return ann

In [4]:
%%time
EXP_NAME2 = 'exp5.1'
hps2 = get_hp(EXP_NAME2)
EXP_NAME = 'exp5'
hps = get_hp(EXP_NAME)

print('Lead: '+str(hps2['LEAD']))
print('Region: '+str(hps2['region'][0])+'N x '+str(hps2['region'][1])+'E')

for kk in np.arange(0,18):
    print('Validation Member: '+str(hps2['val_mems'][kk][0]))
    print('Testing Member: '+str(hps2['test_mems'][kk][0]))
    
    for i_trainmems in range(8):
        print(i_trainmems)
        print('Training Members: '+str(hps2['train_mems'][kk][i_trainmems]))
        
    
        X2train, X2val, _, Y2train, Y2val, _, _ = split_cesm_retrain(trainmems = hps2['train_mems'][kk][i_trainmems],
                                                                  valmem = hps2['val_mems'][kk][0],
                                                                  testmem = hps2['test_mems'][kk][0],
                                                                  months = [11,12,1,2], # months for X (Y+leadtime are accounted for in function)
                                                                  lead = hps2['LEAD'],
                                                                  latpt = hps2['region'][0],  # 45 or 55
                                                                  lonpt = hps2['region'][1]) # 330 or 210
    
        X2val = X2val.reshape((X2val.shape[0],X2val.shape[1]*X2val.shape[2]))
        X2train = X2train.reshape((X2train.shape[0],X2train.shape[1]*X2train.shape[2]))
        input_shape = X2train.shape[1]
    
        # ---------- ANN Hyperparameters ----------
        NLABEL = 2
        N_EPOCHS = 1000
    
        hps = get_hp(EXP_NAME) # need to have this here because .extend overwrites HIDDENS
    
        HIDDENS = hps['HIDDENS']
        BATCH_SIZE = hps['BATCH_SIZE']
        LR_INIT = hps['LR_INIT']
        RIDGE = hps['RIDGE']
        dropout_rate = hps['DROPOUT_RATE']
        PATIENCE = hps['PATIENCE']
        GLOBAL_SEED = hps['GLOBAL_SEED']
        # HIDDENS.extend(ann2HIDDENS)
    
        ann2LR_INIT = hps2['LR_INIT'] 
        ann2BATCH_SIZE = hps2['BATCH_SIZE'] 
        ann2dropout_rate = hps2['DROPOUT_RATE'] 
        ann2RIDGE = hps2['RIDGE'] 
    
        #----- SET UP & TRAIN NN -----
        np.random.seed(GLOBAL_SEED)
        random.seed(GLOBAL_SEED)
        tf.random.set_seed(GLOBAL_SEED)
    
        for SEED in np.arange(5,10):
            tf.keras.backend.clear_session() 
    
            # ------ define NN ------
            ann2 = create_ann2(input_shape = input_shape,
                               output_shape = NLABEL,
                               hiddens=HIDDENS,
                               RIDGE = ann2RIDGE,
                               dropout = True, dropout_rate = ann2dropout_rate)
    
            annfi_name = 'ann_60Ebias_'+EXP_NAME+'.'+str(kk)+'_seed'+str(SEED)+'.h5'
            ann1 = tf.keras.models.load_model(base_dir+'models/kfold/'+annfi_name)
    
            ann2.layers[1].set_weights(ann1.layers[1].get_weights())
    
            # ann2.summary()
    
            # ------ Training Hyperparameters ------
            optimizer = tf.optimizers.Adam(learning_rate = ann2LR_INIT,)
            loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
            metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name="sparse_categorical_accuracy", dtype=None)]                             
    
            # ------ Compile Model -----
            ann2.compile(optimizer = optimizer,
                        loss = loss_func,
                        metrics = metrics)
    
            # ----- Callbacks -----
            ES = tf.keras.callbacks.EarlyStopping(monitor = 'val_sparse_categorical_accuracy', mode = 'auto', min_delta = 0.001,
                                                  patience = PATIENCE, verbose = 0, restore_best_weights = True)
    
            # ------ Train -----
            start_time = time.time()
            history = ann2.fit(X2train,Y2train,
                              validation_data = [X2val,Y2val],
                              batch_size = ann2BATCH_SIZE,
                              epochs = N_EPOCHS,
                              shuffle = True,
                              verbose = 0,
                              callbacks = [ES])
            stop_time = time.time()
            tf.print(f"Elapsed time during fit = {(stop_time - start_time)/60.:.2f} minutes\n")
    
            #----- PLOT THE RESULTS -----
            if kk == 0 and SEED == 0:
                plot_results(
                    history,
                    exp_info=(100, HIDDENS, ann2LR_INIT, ann2BATCH_SIZE, SEED, PATIENCE, ann2RIDGE),
                    showplot=True
                )

                pred1 = np.argmax(ann1.predict(X2val),axis=-1)
                pred2 = np.argmax(ann2.predict(X2val),axis=-1)
    
                print((np.shape(np.where(pred1==Y2val)[0])[0]/np.shape(Y2val)[0])*100) # og network 
                print((np.shape(np.where(pred2==Y2val)[0])[0]/np.shape(Y2val)[0])*100) # first layer weights transferred 
    
            annfi_name = 'ann2_60Ebias_'+EXP_NAME2+'.'+str(kk)+'.'+str(i_trainmems)+'_ann1_'+EXP_NAME+'_seed'+str(SEED)+'.h5'
            ann2.save(base_dir+'models/kfold/'+annfi_name)
                

Lead: 14
Region: 30-60N x 170-240E
Validation Member: 1
Testing Member: 2
0
Training Members: [13]
files do not exsist - loading data & saving
made it!


2024-03-20 09:44:38.782481: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Elapsed time during fit = 0.20 minutes

Elapsed time during fit = 0.13 minutes

Elapsed time during fit = 0.18 minutes

Elapsed time during fit = 0.15 minutes

Elapsed time during fit = 0.15 minutes

1
Training Members: [13, 14]
files do not exsist - loading data & saving
Elapsed time during fit = 0.30 minutes

Elapsed time during fit = 0.17 minutes

Elapsed time during fit = 0.10 minutes

Elapsed time during fit = 0.17 minutes

Elapsed time during fit = 0.17 minutes

2
Training Members: [13, 14, 15]
files do not exsist - loading data & saving
Elapsed time during fit = 0.33 minutes

Elapsed time during fit = 0.32 minutes

Elapsed time during fit = 0.26 minutes

Elapsed time during fit = 0.16 minutes

Elapsed time during fit = 0.36 minutes

3
Training Members: [13, 14, 15, 16]
files do not exsist - loading data & saving
Elapsed time during fit = 0.34 minutes

Elapsed time during fit = 0.38 minutes

Elapsed time during fit = 0.25 minutes

Elapsed time during fit = 0.19 minutes

Elapsed t